In [2]:
! pip install html5lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 2.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from bs4 import BeautifulSoup
import requests

url = "https://www.house.gov/representatives"
text = requests.get(url).text
soup = BeautifulSoup(text)

all_urls = [a['href']
            for a in soup('a')
            if a.has_attr('href')]

print(len(all_urls))

967


We can use Regex Expression

In [3]:
import re 

# Must start wutg http:// or https://
# Must end with .house.gov or .house.gov/

regex = r'^https?://.*\.house\.gov/?$'

# Let's write some tests!
assert re.match(regex,"http://joel.house.gov")
assert re.match(regex,"https://joel.house.gov")
assert re.match(regex,"http://joel.house.gov/")
assert re.match(regex,"https://joel.house.gov/")
assert not re.match(regex,"joel.house.gov")
assert not re.match(regex,"http://joel.house.com")
assert not re.match(regex,"http://joel.house.gov/biography")

In [9]:
# Lets apply the regex
good_urls = [ url for url in all_urls if re.match(regex,url)]
print(len(good_urls))

872


In [11]:
good_urls = list(set(good_urls))
print(len(good_urls))

436


In [8]:
html = requests.get('https://jaypal.house.gov').text
soup = BeautifulSoup(html)

# Use a set as there might be multiple links repeated
links = {a['href'] for a in soup('a') if 'press releases' in a.text.lower()}

print(links)

ConnectionError: HTTPSConnectionPool(host='jaypal.house.gov', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f6325aa65f0>: Failed to resolve 'jaypal.house.gov' ([Errno -2] Name or service not known)"))

In [12]:
from typing import Dict, Set

press_releases : Dict[str, Set[str]] = {} 

for house_url in good_urls:
    html = requests.get(house_url).text
    soup = BeautifulSoup(html,'html5lib')
    pr_links = {a['href'] for a in soup('a') if 'press releases' in a.text.lower()}

    print(f'{house_url} : {pr_links}')
    press_releases[house_url] = pr_links

https://scottpeters.house.gov : {'/press-releases'}
https://schrier.house.gov : {'/media/press-releases'}
https://magaziner.house.gov : {'/media/press-releases'}


https://kiley.house.gov : set()
https://rosendale.house.gov : {'/news/documentquery.aspx?DocumentTypeID=27'}
https://manning.house.gov : {'/media/press-releases'}
https://rouzer.house.gov/ : {'/news/documentquery.aspx?DocumentTypeID=27'}
https://womack.house.gov/ : {'/News/DocumentQuery.aspx?DocumentTypeID=2067'}
https://posey.house.gov/ : {'/News/DocumentQuery.aspx?DocumentTypeID=1487'}
https://gallagher.house.gov : {'/media/press-releases'}
https://cline.house.gov : {'/news/documentquery.aspx?DocumentTypeID=27'}
https://stanton.house.gov/ : {'/press-releases'}
https://houchin.house.gov : {'/media/press-releases'}
https://steube.house.gov/ : {'https://steube.house.gov/category/press-releases/'}
https://bishop.house.gov/ : {'/media-center/press-releases'}
https://bentz.house.gov : {'/media/press-releases'}
https://frankel.house.gov : {'/news/documentquery.aspx?DocumentTypeID=27'}
https://webster.house.gov/ : {'/press-releases'}
https://ramirez.house.gov : {'/media/press-releases'}
http

In [13]:
def paragraph_mentions(text: str, keyword: str) -> bool:
    """ 
    Returns True if a <p> inside tge text mentions {keyword}
    """

    soup = BeautifulSoup(text,'html5lib')
    paragraphs = [ p.get_text() for p in soup('p')]

    return any(keyword.lower() in paragraph.lower() for paragraph in paragraphs)



In [15]:
for house_url , pr_links in press_releases.items():
    for pr_link in pr_links:
        url = f"{house_url}/{pr_link}"
        text = requests.get(url).text

        if paragraph_mentions(text, 'data'):
            print(f"{house_url}")
            break

https://gallagher.house.gov


https://cammack.house.gov
https://fallon.house.gov
https://tokuda.house.gov
https://grothman.house.gov
https://veasey.house.gov
https://dankildee.house.gov
https://cartwright.house.gov
https://laurellee.house.gov
https://beyer.house.gov
https://deluzio.house.gov
https://duarte.house.gov
https://curtis.house.gov/
